<a href="https://colab.research.google.com/github/blancavazquez/CursoDatosMasivosII/blob/2023-I/FPGrowth_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Algoritmo FP-Growth
El objetivo de esta libreta es identificar las reglas de asociación a través del algoritmo de FP-Growth.
Para esta libreta se usará la biblioteca de [mlxtend](http://rasbt.github.io/mlxtend/).


Conceptos importantes a recordar:


*   Soporte: calcula la frecuencia de un elemento
*   Confianza: es la probabilidad de que ocurra el consecuente dado el antecendete, se emplea en las reglas de asociación. Por ejemplo: If (cebollas, vegetales) then (carne).
*   Lift: compara la frecuencia observada con la frecuencia esperada simplemente por azar. Para calcular lift:

$$
lift = \frac{soporte (union(X,Y))}{soporte(X)*soporte(Y)}
$$


En otras palabras, Lift refleja el aumento de la probabilidad de que ocurra el consecuente, cuando nos enteramos de que ocurrió el antecedente. 


*   Si lift es mayor a 1 indica que ese conjunto aparece una cantidad de veces superior a lo esperado bajo condiciones de independencia (hay suficiente evidencia que la regla no apareció al azar).
*   Si lift es igual 1 indica que ese conjunto aparece una cantidad de veces acorde a lo esperado bajo condiciones de independencia.
*   Si lift es menor a 0 indica que ese conjunto aparece una cantidad de veces inferior a lo esperado bajo condiciones de independencia.



In [1]:
#Instalando la biblioteca de mlxtend
%pip install mlxtend --upgrade
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
data = [['Leche', 'Queso', 'Café', 'Bolillos', 'Huevos', 'Yogurt'],
           ['Mantequilla', 'Queso', 'Café', 'Bolillos', 'Huevos', 'Yogurt'],
           ['Leche', 'Manzanas', 'Bolillos', 'Huevos'],
           ['Leche', 'Mantequilla', 'Mermelada', 'Bolillos', 'Yogurt'],
           ['Mermelada', 'Queso', 'Queso', 'Bolillos', 'Aguacates', 'Huevos']]

In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

tencoder = TransactionEncoder() #Convert item lists into transaction data

In [4]:
te_data = tencoder.fit(data).transform(data)
te_data #genera un array de tipo binario

array([[False,  True,  True,  True,  True, False, False, False,  True,
         True],
       [False,  True,  True,  True, False,  True, False, False,  True,
         True],
       [False,  True, False,  True,  True, False,  True, False, False,
        False],
       [False,  True, False, False,  True,  True, False,  True, False,
         True],
       [ True,  True, False,  True, False, False, False,  True,  True,
        False]])

In [5]:
df = pd.DataFrame(te_data, columns=tencoder.columns_)
df

,Aguacates,Bolillos,Café,Huevos,Leche,Mantequilla,Manzanas,Mermelada,Queso,Yogurt
0,False,True,True,True,True,False,False,False,True,True
1,False,True,True,True,False,True,False,False,True,True
2,False,True,False,True,True,False,True,False,False,False
3,False,True,False,False,True,True,False,True,False,True
4,True,True,False,True,False,False,False,True,True,False


## Ejecutando el algoritmo de fpgrowth

In [6]:
#Carga del algoritmo de FPGrowth
from mlxtend.frequent_patterns import fpgrowth
fpgrowth(df, min_support=0.6)

,support,itemsets
0,1.0,(1)
1,0.8,(3)
2,0.6,(9)
3,0.6,(8)
4,0.6,(4)
5,0.8,"(1, 3)"
6,0.6,"(9, 1)"
7,0.6,"(8, 3)"
8,0.6,"(8, 1)"
9,0.6,"(8, 1, 3)"


In [7]:
fpgrowth(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,1.0,(Bolillos)
1,0.8,(Huevos)
2,0.6,(Yogurt)
3,0.6,(Queso)
4,0.6,(Leche)
5,0.8,"(Bolillos, Huevos)"
6,0.6,"(Bolillos, Yogurt)"
7,0.6,"(Queso, Huevos)"
8,0.6,"(Bolillos, Queso)"
9,0.6,"(Bolillos, Queso, Huevos)"


## Algoritmo Apriori versus FPGrowth

In [8]:
#Carga de la bilioteca para ejecutar algoritmo Apriori
from mlxtend.frequent_patterns import apriori

In [9]:
%timeit -n 100 -r 10 apriori(df, min_support=0.6)

6.12 ms ± 1.76 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [10]:
%timeit -n 100 -r 10 fpgrowth(df, min_support=0.6)

1.87 ms ± 715 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


## Algoritmo FP-Growth con Spark

In [11]:
!pip install pyspark
from pyspark.ml.fpm import FPGrowth
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext("local")
spark = SparkSession(sc)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
transacciones = spark.createDataFrame([
    (0, ["Leche", "Queso", "Café", "Bolillos", "Huevos", "Yogurt"]),
    (1, ["Mantequilla", "Queso", "Café", "Bolillos", "Huevos", "Yogurt"]),
    (2, ["Leche", "Manzanas", "Bolillos", "Huevos"]),
    (3, ["Leche", "Mantequilla", "Mermelada", "Bolillos", "Yogurt"]),
    (4, ["Mermelada", "Queso", "Bolillos", "Aguacates", "Huevos"])
], ["id", "items"])

In [13]:
"Declaramos el algoritmo"
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(transacciones)

In [14]:
"Visualizando los elementos más frecuentes"
model.freqItemsets.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|            [Yogurt]|   3|
|  [Yogurt, Bolillos]|   3|
|             [Leche]|   3|
|   [Leche, Bolillos]|   3|
|            [Huevos]|   4|
|  [Huevos, Bolillos]|   4|
|             [Queso]|   3|
|     [Queso, Huevos]|   3|
|[Queso, Huevos, B...|   3|
|   [Queso, Bolillos]|   3|
|          [Bolillos]|   5|
+--------------------+----+



In [15]:
"Visualizando las reglas de asociación"
model.associationRules.show()

+------------------+----------+----------+----+-------+
|        antecedent|consequent|confidence|lift|support|
+------------------+----------+----------+----+-------+
|          [Yogurt]|[Bolillos]|       1.0| 1.0|    0.6|
|   [Queso, Huevos]|[Bolillos]|       1.0| 1.0|    0.6|
|           [Leche]|[Bolillos]|       1.0| 1.0|    0.6|
|        [Bolillos]|  [Yogurt]|       0.6| 1.0|    0.6|
|        [Bolillos]|   [Leche]|       0.6| 1.0|    0.6|
|        [Bolillos]|  [Huevos]|       0.8| 1.0|    0.8|
|        [Bolillos]|   [Queso]|       0.6| 1.0|    0.6|
| [Queso, Bolillos]|  [Huevos]|       1.0|1.25|    0.6|
|[Huevos, Bolillos]|   [Queso]|      0.75|1.25|    0.6|
|          [Huevos]|[Bolillos]|       1.0| 1.0|    0.8|
|          [Huevos]|   [Queso]|      0.75|1.25|    0.6|
|           [Queso]|  [Huevos]|       1.0|1.25|    0.6|
|           [Queso]|[Bolillos]|       1.0| 1.0|    0.6|
+------------------+----------+----------+----+-------+



In [16]:
"""La función de transform, examina los elementos de entrada,
contra todas las reglas de asociación, indicando los
consecuentes como predicción.

""" 
model.transform(transacciones).show()

+---+--------------------+---------------+
| id|               items|     prediction|
+---+--------------------+---------------+
|  0|[Leche, Queso, Ca...|             []|
|  1|[Mantequilla, Que...|        [Leche]|
|  2|[Leche, Manzanas,...|[Yogurt, Queso]|
|  3|[Leche, Mantequil...|[Huevos, Queso]|
|  4|[Mermelada, Queso...|[Yogurt, Leche]|
+---+--------------------+---------------+

